In [21]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import datetime
from os import path, environ

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import ipyvolume as ipv

from thermo import CONSTANTS
# source file, see docs/5-dataset.md for info on field names
Chiller1File = path.join(environ['DATADIR'], 'EngineeringScienceBuilding', 'Chillers.csv')
plot_path = path.join('..', 'docs', 'img')

In [2]:
# Read pre-processed data
df = pd.read_csv(Chiller1File, index_col='Time', parse_dates=True, dtype=float)

# Correlations

## Fan speed & fan power

*Hypothesis*: Fan power depends on ambient temperature `TempAmbient`, relative humidity `PerHumidity`, and fan speed setting `PerFreqFan[A | B]`.

*Result*: Very little relationship from current data. Primarily because fan speed is ~100%. Need more variation.

In [7]:
# Data
X = df.loc[:, ('TempAmbient', 'PerHumidity', 'PerFreqFanA')]
Y = df.loc[:, 'PowFanA']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1)

In [8]:
# Pipeline
scaler = MinMaxScaler()        # scale to [0-1] range
regressor = LinearRegression() # regression model
reg = Pipeline([('scaler', scaler), ('regressor', regressor)])

In [9]:
reg.fit(Xtrain, Ytrain)
reg.score(Xtest, Ytest)

0.1107982030593837

In [ ]:
f = plt.figure(figsize=(8,8))
ax = f.add_subplot(111, projection='3d')
ax.scatter3D(X.iloc[:, 0], X.iloc[:, 1], Y, c=Y)
ax.set_xlabel('Ambient Temperature (K)')
ax.set_ylabel('Relative Humidity')
ax.set_zlabel('Fan Power (W)')
plt.savefig(path.join(plot_path, '7-fan-power-vs-temp-humidity.png'))

# Clusters

## Temperature

In [12]:
X = df.loc[:, ('TempAmbient', 'TempWetBulb')]
X['DeltaTemp'] = df['TempCondOut'] - df['TempCondIn']
labels = DBSCAN().fit_predict(X)

In [ ]:
ipv.clear()
cmap = plt.cm.Accent(labels)
f = ipv.scatter(X['TempAmbient'], X['DeltaTemp'], X['TempWetBulb'], color=cmap,
            size=10, marker='point_2d')
ipv.xyzlim(280, 310)
ipv.ylim(0, 10)
ipv.xyzlabel('TempAmbient', 'DeltaTemp', 'TempWetBulb')
ipv.show()

In [14]:
def view(fig, n, frac):
    ipv.view(360*frac)
ipv.movie(path.join(plot_path, '7-ct-temp-clusters.gif'), view, fps=12, frames=36)

Output()

![](../docs/img/7-ct-temp-clusers.gif)

## Power

In [15]:
X = df.loc[:, ('PowConP', 'PowFanA', 'PowFanB')]
labels = DBSCAN().fit_predict(X)

In [ ]:
ipv.clear()
cmap = plt.cm.Accent(labels)
f = ipv.scatter(X['PowConP'], X['PowFanA'], X['PowFanB'], color=cmap,
            size=10, marker='point_2d')
# ipv.xyzlim(280, 310)
ipv.xyzlabel('PowConP', 'PowFanA', 'PowFanB')
ipv.show()

In [ ]:
def view(fig, n, frac):
    ipv.view(360*frac)
ipv.movie(path.join(plot_path, '../docs/img/7-ct-power-clusters.gif'), view, fps=12, frames=36)